In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install openimages numpy pandas scikit-learn opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.8 MB/s eta 0:00:00


In [3]:
import os
import random
import cv2
import numpy as np
from openimages.download import download_images

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def load_images_from_folder(folder_path, num_images):
    images = []
    random_files = random.sample(os.listdir(folder_path), num_images)
    for filename in random_files:
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

def test_model(model, image_path):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, (64, 64))  # Resize the image to match the input size during training
        img = img.reshape(1, -1) / 255.0  # Flatten and normalize the image
        prediction = model.predict(img)
        if prediction[0] == 1:
            return "calisthenics_spot"
        else:
            return "not_calisthenics_spot"
    else:
        return "Invalid image path or format"

# Set the paths for calisthenics_spot and not_calisthenics_spot folders
calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/calisthenics_spot"
not_calisthenics_folder = "./not_calisthenics_spot"

# Load images from the calisthenics_spot folder
num_calisthenics_images = 6552
calisthenics_images = load_images_from_folder(calisthenics_folder, num_calisthenics_images)

# Download random images from the Open Images V6 dataset for the not_calisthenics_spot class
num_not_calisthenics_images = 6552
class_label = "not_calisthenics_spot"
download_images(dataset='validation', limit=num_not_calisthenics_images, annotations=class_label,
                folder=not_calisthenics_folder)

# Load images from the not_calisthenics_spot folder
not_calisthenics_images = load_images_from_folder(not_calisthenics_folder, num_not_calisthenics_images)

# Create labels for the data
calisthenics_labels = np.ones(num_calisthenics_images)
not_calisthenics_labels = np.zeros(num_not_calisthenics_images)

# Combine the data and labels for the model
X = np.array(calisthenics_images + not_calisthenics_images)
y = np.concatenate((calisthenics_labels, not_calisthenics_labels))

# Flatten and normalize the images
X = X.reshape(X.shape[0], -1)
X = X.astype('float32') / 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

FileNotFoundError: ignored